In [ ]:
# Importation des bibliothèques nécessaires
import torch
import matplotlib.pyplot as plt

# Définition de la classe LinearClassification pour faire de la classification linéaire avec softmax et cross-entropy
class LinearClassification:
    def __init__(self, x, w, y, learning_rate=0.01):
        # Initialisation des poids avec gradient
        self.w = torch.tensor(w, requires_grad=True, dtype=torch.float32)
        # Stockage des données et des labels
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.learning_rate = learning_rate

    # Calcul de la prédiction (produit matriciel)
    def forward(self, x):
        return x @ self.w

    # Fonction softmax pour convertir les logits en probabilités
    def softmax(self, res):
        res = torch.exp(res) / torch.sum(torch.exp(res), dim=1, keepdim=True)
        return res

    # Fonction de perte cross-entropy
    def loss_function(self, logits, ytrue):
        ypred = self.softmax(logits)
        return -torch.mean(torch.sum(ytrue * torch.log(ypred), dim=1, keepdim=True))

    # Fonction de perte avec régularisation L2
    def L2_loss(self, logits, ytrue, l):
        ypred = self.softmax(logits)
        return -torch.mean(torch.sum(ytrue * torch.log(ypred), dim=1, keepdim=True)) + l * torch.sum(self.w ** 2)

    # Séparation des données en train et validation
    def split_data(self):
        indices = torch.randperm(len(self.x))
        size = int(len(self.x) * 0.7)
        x = self.x[indices]
        y = self.y[indices]
        x_train = x[:size]
        y_train = y[:size]
        x_val = x[size:]
        y_val = y[size:]
        return x_train, y_train, x_val, y_val

    # Génération des folds pour la cross-validation
    def cross_validation(self, k):
        fold_len = len(self.x) // k
        indices = torch.randperm(len(self.x))
        x = self.x[indices]
        y = self.y[indices]
        x_folds = []
        y_folds = []
        for i in range(k):
            x_folds.append(x[fold_len * i: fold_len * (i + 1)])
            y_folds.append(y[fold_len * i: fold_len * (i + 1)])
        return x_folds, y_folds

    # Calcul du gradient et mise à jour des poids
    def backward_propagation(self, loss):
        loss.backward()
        with torch.no_grad():
            self.w -= self.learning_rate * self.w.grad
        self.w.grad.zero_()

    # Batch Gradient Descent
    def batch_gd(self, epochs, x_train, y_train, x_val, y_val, l2=False, lambda_l2=0.1):
        losses = []
        val_losses = []
        for _ in range(epochs):
            train_prediction = self.forward(x_train)
            val_prediction = self.forward(x_val)
            if l2:
                loss = self.L2_loss(train_prediction, y_train, lambda_l2)
            else:
                loss = self.loss_function(train_prediction, y_train)
            self.backward_propagation(loss)
            val_losses.append(self.loss_function(val_prediction, y_val).item())
            losses.append(loss.item())
        return losses, val_losses

    # Mini-Batch Stochastic Gradient Descent
    def minibatch_SGD(self, epochs, x_train, y_train, x_val, y_val, batch_size, l2=False, lambda_l2=0.1):
        losses = []
        val_losses = []
        num_batches = len(x_train) // batch_size
        for _ in range(epochs):
            indices = torch.randperm(len(x_train))
            x_train = x_train[indices]
            y_train = y_train[indices]
            epoch_losses = []
            for i in range(num_batches):
                start = i * batch_size
                end = start + batch_size
                x_batch = x_train[start:end]
                y_batch = y_train[start:end]
                train_prediction = self.forward(x_batch)
                if l2:
                    loss = self.L2_loss(train_prediction, y_batch, lambda_l2)
                else:
                    loss = self.loss_function(train_prediction, y_batch)
                epoch_losses.append(loss.item())
                self.backward_propagation(loss)
            losses.append(sum(epoch_losses) / len(epoch_losses))
            val_prediction = self.forward(x_val)
            val_losses.append(self.loss_function(val_prediction, y_val).item())
        return losses, val_losses

    # Stochastic Gradient Descent
    def SGD(self, epochs, x_train, y_train, x_val, y_val, l2=False, lambda_l2=0.1):
        losses = []
        val_losses = []
        for _ in range(epochs):
            indices = torch.randperm(len(x_train))
            x_train = x_train[indices]
            y_train = y_train[indices]
            epoch_losses = []
            for i in range(len(x_train)):
                xi = x_train[i].unsqueeze(0)
                yi = y_train[i].unsqueeze(0)
                train_prediction = self.forward(xi)
                if l2:
                    loss = self.L2_loss(train_prediction, yi, lambda_l2)
                else:
                    loss = self.loss_function(train_prediction, yi)
                epoch_losses.append(loss.item())
                self.backward_propagation(loss)
            losses.append(sum(epoch_losses) / len(epoch_losses))
            val_prediction = self.forward(x_val)
            val_losses.append(self.loss_function(val_prediction, y_val).item())
        return losses, val_losses

    # Évaluation de l'accuracy
    def evaluate_accuracy(self, ypred, ytrue):
        predicted_classes = torch.argmax(ypred, dim=1)
        true_classes = torch.argmax(ytrue, dim=1)
        return (predicted_classes == true_classes).sum().item() / len(ypred)

    # Visualisation des courbes de perte
    def plot(self, losses, val_losses, epochs):
        plt.plot(range(epochs), losses, label="train_loss")
        plt.plot(range(epochs), val_losses, label="val_loss")
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Évolution de la perte')
        plt.legend()
        plt.show()
